## Combining year templates

This notebook will take templates (created from the create_cards notebook) and combine them over the different years. It will also create "combined" cards, which technically are not "combined" cards, as this is exactly not just a combination of cards, but really a combination / hadding of years.

In [1]:
import uproot
import hist
import numpy as np
from core import *

In [2]:
# defining samples and stuff
masspoints = [700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2250, 2500, 2750, 3000]

years = ["UL16preVFP", "UL16postVFP", "UL17", "UL18"]
channels = ["ele", "mu"]   # splitting electron and muon channel
spins = ["", "_Spin32"]
regions = ["SR", "VR"]

samples = ["TstarTstar{}_M-{}", "TTbar", "ST", "datadriven"] # first argument will be filled in the lööp
MC_samples = samples[:-1] # datadriven is treated separately

inputdir = "cards"
outputdir = "cards_combined"

In [3]:
# defining systematic treatment
# this needs to be different depending on the type of syst, and correlation

MC_uncertainties = { # name and whether its correlated
    "pu" : True,
    "prefiring": False, 
    "btagging_hf": True,
    "btagging_hfstats1": False,
    "btagging_hfstats2": False,
    "btagging_lf": True,
    "btagging_lfstats1": False,
    "btagging_lfstats2": False,
    "btagging_cferr1": True,
    "btagging_cferr2": True,
    "sfelec_id": True,
    "sfelec_reco": True,
    "sfelec_trigger": False,
    "sfmu_id": False,
    "sfmu_iso": False,
    "sfmu_trigger": False,
    "topPt": True,
    "decorrelation": True, # take care that this does not work for signal!
    "JEC": False,
    "JER": False,
    "btagYield" : True,
}

datadriven_uncertainties = {
    "datadrivenFitFunction" : True,
    "datadrivenFitStat" : True,
    "btagging_total" : True,    
}

sample_specific = {
    "scale_{}" : True,
    "PDF_{}" : True,
}

In [4]:
# we'll want to create a single root file per signal point, spin, region and channel
# channel combination then happens in a secondary step!

for masspoint in masspoints:    
    for spin in spins:
        
        this_samples = samples.copy()
        this_samples[0] = this_samples[0].format(spin, masspoint)
        
        for channel in channels:
            for region in regions:
            
                print(f"Processing masspoint {masspoint}, spin {spin}, channel {channel}, region {region}")
                
                # defining the file name, which will stay the same
                fname = f"{masspoint}_pt_ST_{channel}_{region}{spin}.root"

                with uproot.recreate(outputdir + "/" + fname) as outf:
                    
                    # combine data
                    hists_all = []
                    for year in years: # looping over years and getting all entries
                        with uproot.open(inputdir + "/" + year + "/" + fname) as inf:
                            hists_all.append(inf["data_obs"].to_hist())
                    hist_sum = sum(hists_all) # summing up the years
                    outf["data_obs"] = hist_sum# writing to root file

                    for sample in this_samples:

                        ### first, do nominal
                        hists_all = []
                        for year in years:
                            with uproot.open(inputdir + "/" + year + "/" + fname) as inf:
                                hists_all.append(inf[sample].to_hist())
                        hist_sum = sum(hists_all)
                        outf[sample] = hist_sum

                        # now lets do shape uncertanties
                        # differentiating by sample
                        if sample == "datadriven": this_uncertainties = datadriven_uncertainties.copy()
                        else:
                            this_uncertainties = MC_uncertainties.copy()
                            for specific, correlation in sample_specific.items():
                                this_uncertainties[specific.format(sample)] = correlation

                        for uncertainty, correlation in this_uncertainties.items():
                            for variation in ["Up", "Down"]:

                                if "TstarTstar" in sample and "decorrelation" in uncertainty: continue # no decorrrelation for signal

                                if correlation:
                                    # fully correlated is easy, just do same as nominal

                                    corrstring = "UL16preVFPUL16postVFPUL17UL18"

                                    hists_all = []
                                    for year in years:
                                        with uproot.open(inputdir + "/" + year + "/" + fname) as inf:
                                            hists_all.append(inf[sample+"_"+uncertainty+"_"+corrstring+variation].to_hist())
                                    hist_sum = sum(hists_all)
                                    if("btagYield" in uncertainty): outf[sample+"_"+uncertainty+"_"+channel+"_"+corrstring+variation] = hist_sum
                                    else: outf[sample+"_"+uncertainty+"_"+corrstring+variation] = hist_sum

                                else:
                                    # partially correlated needs the variation from one year,
                                    # plus the nominal from all others

                                    for year in years:

                                        corrstring = year

                                        hists_all = []
                                        with uproot.open(inputdir + "/" + year + "/" + fname) as inf:
                                            hists_all.append(inf[sample+"_"+uncertainty+"_"+corrstring+variation].to_hist())
                                        for other_year in years:
                                            with uproot.open(inputdir + "/" + other_year + "/" + fname) as inf:
                                                if year == other_year: continue
                                                hists_all.append(inf[sample].to_hist())
                                        hist_sum = sum(hists_all)
                                        if("btagYield" in uncertainty): outf[sample+"_"+uncertainty+"_"+channel+"_"+corrstring+variation] = hist_sum
                                        else: outf[sample+"_"+uncertainty+"_"+corrstring+variation] = hist_sum
                                        #outf[sample+"_"+uncertainty+"_"+corrstring+variation] = hist_sum
                                    

Processing masspoint 700, spin , channel ele, region SR
Processing masspoint 700, spin , channel ele, region VR
Processing masspoint 700, spin , channel mu, region SR
Processing masspoint 700, spin , channel mu, region VR
Processing masspoint 700, spin _Spin32, channel ele, region SR
Processing masspoint 700, spin _Spin32, channel ele, region VR
Processing masspoint 700, spin _Spin32, channel mu, region SR
Processing masspoint 700, spin _Spin32, channel mu, region VR
Processing masspoint 800, spin , channel ele, region SR
Processing masspoint 800, spin , channel ele, region VR
Processing masspoint 800, spin , channel mu, region SR
Processing masspoint 800, spin , channel mu, region VR
Processing masspoint 800, spin _Spin32, channel ele, region SR
Processing masspoint 800, spin _Spin32, channel ele, region VR
Processing masspoint 800, spin _Spin32, channel mu, region SR
Processing masspoint 800, spin _Spin32, channel mu, region VR
Processing masspoint 900, spin , channel ele, region SR


Processing masspoint 3000, spin , channel ele, region VR
Processing masspoint 3000, spin , channel mu, region SR
Processing masspoint 3000, spin , channel mu, region VR
Processing masspoint 3000, spin _Spin32, channel ele, region SR
Processing masspoint 3000, spin _Spin32, channel ele, region VR
Processing masspoint 3000, spin _Spin32, channel mu, region SR
Processing masspoint 3000, spin _Spin32, channel mu, region VR


In [5]:
# copied from core.py
# todo replace by something central
# or not

def pad(s, n_pad):
    s = str(s)
    n_pad = n_pad - len(s)
    return s + n_pad * ' '

# defining some global constants
N1 = 79
N2 = 8
N3 = 25

autoMCstatsParam = 0

def write_block_header(f, block_name: str):
    f.write(f"# {block_name.capitalize()}\n")
    f.write(N1 * "-" + "\n")

def write_parameters(f, n_procs, region, fname):
    write_block_header(f, "parameters")
    f.write(f"imax 1\njmax {n_procs-1}\nkmax *\n")
    f.write(f"shapes * {region} {fname} "
            f"$PROCESS $PROCESS_$SYSTEMATIC\n\n")

def write_channels(f, region):
    write_block_header(f, "channels")
    f.write(f"bin          {region}\n")
    f.write("observation  -1\n\n")
    
def write_processes(f, processes, region):
    padded_processes = [pad(x, N3) for x in processes]
    padded_ids = [pad(i , N3) for i in range(len(processes))]
    write_block_header(f, "processes")
    f.write(pad("bin", N1 + N2) + len(processes) * pad(region, N3) + "\n")
    f.write(pad("process", N1 + N2) + "".join(padded_processes) + "\n")
    f.write(pad("process", N1 + N2) + "".join(padded_ids) + "\n")
    f.write(pad("rate", N1 + N2) + len(processes) * pad("-1", N3))
    f.write("\n\n")
    
def write_lumi(f, processes):
    write_block_header(f, "systematics")
    f.write(pad("lumi_13TeV_total", N1) + pad("lnN", 8))
    for process in processes:
        if not process == "datadriven":
            f.write(pad(1.016, N3))
        else:
            f.write(pad("-", N3))
    f.write("\n")

In [6]:
# now lets also write the cards!

for masspoint in masspoints:    
    for spin in spins:
        
        this_samples = samples.copy()
        this_samples[0] = this_samples[0].format(spin, masspoint)
        
        for channel in channels:
            for region in regions:
                
                print(f"Processing masspoint {masspoint}, spin {spin}, channel {channel}, region {region}")

                with open(f"{outputdir}/{masspoint}_pt_ST_{channel}_{region}{spin}.dat", 'w') as card:
                    
                    # write the header
                    write_parameters(card, len(samples), region, f"{masspoint}_pt_ST_{channel}_{region}{spin}.root")
                    write_channels(card, region)
                    write_processes(card, this_samples, region)
                    write_lumi(card, this_samples)
                    
                    for uncertainty, correlation in MC_uncertainties.items():
                        
                        if correlation:
                            corrstring = "UL16preVFPUL16postVFPUL17UL18"
                            if "btagYield" in uncertainty: line = pad(uncertainty + "_" + channel + "_" + corrstring, N1)
                            else: line = pad(uncertainty + "_" + corrstring, N1)
                            line += pad("shape", 8)
                            
                            for sample in samples:
                                if sample == "datadriven" or ("TstarTstar" in sample and "decorrelation" in uncertainty):
                                    line += pad("-", N3)
                                else:
                                    line += pad(1, N3)
                                
                            line += "\n"
                            card.write(line)
                        else:
                            for year in years:
                                corrstring = year
                                if "btagYield" in uncertainty: line = pad(uncertainty + "_" + channel + "_" + corrstring, N1)
                                else: line = pad(uncertainty + "_" + corrstring, N1)
                                #line = pad(uncertainty + "_" + corrstring, N1)
                                line += pad("shape", 8)

                                for sample in samples:
                                    if sample == "datadriven" or ("TstarTstar" in sample and "decorrelation" in uncertainty):
                                        line += pad("-", N3)
                                    else:
                                        line += pad(1, N3)

                                line += "\n"
                                card.write(line)
                        
                        
                    for uncertainty, correlation in datadriven_uncertainties.items():
                        
                        if correlation:
                            corrstring = "UL16preVFPUL16postVFPUL17UL18"
                            if "btagYield" in uncertainty: line = pad(uncertainty + "_" + channel + "_" + corrstring, N1)
                            else: line = pad(uncertainty + "_" + corrstring, N1)
                            #line = pad(uncertainty + "_" + corrstring, N1)
                            line += pad("shape", 8)
                            
                            for sample in samples:
                                if sample == "datadriven":
                                    line += pad(1, N3)
                                else:
                                    line += pad("-", N3)
                                
                            line += "\n"
                            card.write(line)
                        else:
                            raise Exception("not implemented!!!") # not needed
                        
                        
                    for uncertainty, correlation in sample_specific.items():
                        
                        if correlation:                            
                            for sample in samples:
                                if sample == "datadriven": continue
                                corrstring = "UL16preVFPUL16postVFPUL17UL18"
                                if "TstarTstar" in sample:
                                    line = pad(uncertainty.format(sample.format(spin, masspoint)) + "_" + corrstring, N1)
                                else:
                                    line = pad(uncertainty.format(sample) + "_" + corrstring, N1)
                                line += pad("shape", 8)
                                for other_sample in samples:
                                    if sample == other_sample:
                                        line += pad(1, N3)
                                    else:
                                        line += pad("-", N3)
                                line += "\n"
                                card.write(line)
                        else:
                            raise Exception("not implemented!!!") # not needed
                            
                            
                    card.write("* autoMCStats {}".format(autoMCstatsParam))

Processing masspoint 700, spin , channel ele, region SR
Processing masspoint 700, spin , channel ele, region VR
Processing masspoint 700, spin , channel mu, region SR
Processing masspoint 700, spin , channel mu, region VR
Processing masspoint 700, spin _Spin32, channel ele, region SR
Processing masspoint 700, spin _Spin32, channel ele, region VR
Processing masspoint 700, spin _Spin32, channel mu, region SR
Processing masspoint 700, spin _Spin32, channel mu, region VR
Processing masspoint 800, spin , channel ele, region SR
Processing masspoint 800, spin , channel ele, region VR
Processing masspoint 800, spin , channel mu, region SR
Processing masspoint 800, spin , channel mu, region VR
Processing masspoint 800, spin _Spin32, channel ele, region SR
Processing masspoint 800, spin _Spin32, channel ele, region VR
Processing masspoint 800, spin _Spin32, channel mu, region SR
Processing masspoint 800, spin _Spin32, channel mu, region VR
Processing masspoint 900, spin , channel ele, region SR


Processing masspoint 3000, spin _Spin32, channel ele, region VR
Processing masspoint 3000, spin _Spin32, channel mu, region SR
Processing masspoint 3000, spin _Spin32, channel mu, region VR
